# Import Required Libraries
Import the necessary libraries, including pathlib, matplotlib, os, and cgshop2025_pyutils.

   >[!NOTE]
   >
   >Notebook requires the `ipywidgets` and `cgshop2025_pyutils` library for python, it is necessary to install with next line command:
   >
   >```bash
   >pip install ipywidgets
   >pip install cgshop2025_pyutils
   >```

In [1]:
# Import the necessary libraries
from pathlib import Path
from matplotlib import pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy.spatial import ConvexHull
from user_interface import ui_setup as ui

# Import specific classes and functions from cgshop2025_pyutils
from cgshop2025_pyutils import (
    DelaunayBasedSolver,
    InstanceDatabase,
    ZipSolutionIterator,
    ZipWriter,
    verify,
    visualization
)

# Locate the Instances
Locate the instances using the InstanceDatabase class from cgshop2025_pyutils.

In [2]:
# Locate the instances using the InstanceDatabase class from cgshop2025_pyutils
idb = InstanceDatabase("example_instances/")

# Display the number of instances found
print(f"Number of instances found: {sum(1 for e in idb)}")

Number of instances found: 40


# Delete Existing Solution Zip File
Check if the solution zip file already exists and delete it if it does.

In [3]:
# Check if the solution zip file already exists and delete it if it does
if Path("example_solutions.zip").exists():
    Path("example_solutions.zip").unlink()
    print("Existing solution zip file deleted.")
else:
    print("No existing solution zip file found.")

Existing solution zip file deleted.


# Compute Solutions for All Instances
Compute solutions for all instances using the provided DelaunayBasedSolver and store them in a list.

In [4]:
# Compute solutions for all instances using the provided DelaunayBasedSolver and store them in a list
solutions = []  # Initialize an empty list to store solutions

# Iterate over each instance in the InstanceDatabase
for instance in idb:
    uid = instance.instance_uid  # Get the unique identifier for the instance
    points_x = instance.points_x  # Get the x-coordinates of the points in the instance
    points_y = instance.points_y  # Get the y-coordinates of the points in the instance
    num_points = instance.num_points  # Get the number of points in the instance
    
    # Print instance details for debugging purposes
    print(f"Instance UID: {uid}")
    print(f"Points X: {points_x}")
    print(f"Points Y: {points_y}")
    print(f"Number of Points: {num_points}")
    
    # Create a solver object for the instance using DelaunayBasedSolver
    solver = DelaunayBasedSolver(instance)
    
    # Compute the solution for the instance
    solution = solver.solve()
    
    # Append the computed solution to the solutions list
    solutions.append(solution)


Instance UID: cgshop2025_examples_simple-polygon-exterior-20_60_5da99fdb
Points X: [5446, 3393, 2803, 2828, 2510, 2228, 2941, 2635, 549, 1615, 1962, 2174, 2173, 2300, 1390, 424, 1106, 3366, 5560, 5981, 6788, 7509, 6907, 7917, 8925, 8719, 9284, 9515, 8495, 8184, 6765, 4401, 3033, 3425, 3734, 2937, 3581, 3840, 5250, 5665, 4681, 5370, 6548, 6995, 8308, 9536, 9214, 9431, 9038, 9277, 9783, 9121, 7880, 7816, 8102, 7588, 7170, 6512, 6641, 6387]
Points Y: [9407, 9383, 9271, 9987, 9506, 9388, 8700, 8057, 7771, 6806, 4788, 3910, 3316, 3040, 3473, 3012, 1172, 964, 1488, 89, 1212, 1038, 256, 164, 118, 731, 1197, 1673, 1653, 2039, 3390, 3766, 3544, 3832, 4365, 7028, 7931, 8281, 7272, 6834, 6655, 6379, 5206, 5038, 4889, 3894, 4554, 5868, 6471, 7858, 8845, 8974, 9061, 7585, 7025, 6704, 7437, 8695, 9017, 9909]
Number of Points: 60
Instance UID: cgshop2025_examples_ortho_20_b099d1fe
Points X: [0, 1000000, 1000000, 998492, 998492, 911972, 911972, 841555, 841555, 807405, 807405, 755435, 755435, 600102, 6

# Write Solutions to Zip File
Write the computed solutions to a new zip file using the ZipWriter class from cgshop2025_pyutils.

In [5]:
# Write the computed solutions to a new zip file using the ZipWriter class from cgshop2025_pyutils
with ZipWriter("example_solutions.zip") as zw:
    for solution in solutions:
        zw.add_solution(solution)
    print("Solutions written to example_solutions.zip")

Solutions written to example_solutions.zip


Run below code to update with the selected UID 

# Plot Solutions
Plot the solutions using matplotlib and the visualization module from cgshop2025_pyutils.

In [6]:
# Get first solution for interactive controls
sol_edges = []
for solution in ZipSolutionIterator("example_solutions.zip"):
    sol_edges = solution.edges
    break

# Get input
for instance in idb:
    first_instance = instance.instance_uid
    break

# Init user interface widgets
uid_list,edges_list = ui.set_ui_elements_solution(idb,sol_edges)

# Interactive plot solutions
prev_uid = None
def interactive_plot_solutions(sel_uid,sel_edge):
    # Init plot
    plt.ioff()
    fig,axs = plt.subplots()

    # UID change detector
    global edges_list
    global prev_uid
    global sol_edges
    uid_changed = False
    if sel_uid != prev_uid:
        uid_changed = True
        prev_uid = sel_uid
        print("uid_change")

    # Find instance with uid
    uid_found = False
    for instance in idb:
        if(instance.instance_uid == sel_uid):
            uid_found = True
            points_x = instance.points_x
            points_y = instance.points_y
            # Plot instance points
            #visualization.plot_instance(axs,instance) 
            break

    if uid_found == False : 
        print("uid not found")
    
    # Find solution with iud
    uid_found = False
    sol_edges = []
    for solution in ZipSolutionIterator("example_solutions.zip"):
        if(solution.instance_uid == sel_uid):
            uid_found = True
            sol_edges = solution.edges

    # Update edges list for selection widget
    edges_list,sel_edge = ui.update_edges_list(uid_changed,edges_list,sol_edges,sel_edge)

    # Plot the edges of the solution
    if uid_found:
        for edge in sol_edges:
            axs.plot([points_x[edge[0]], points_x[edge[1]]], [points_y[edge[0]], points_y[edge[1]]], 'r-')
        if sel_edge != None:
            axs.plot([points_x[sel_edge[0]], points_x[sel_edge[1]]], [points_y[sel_edge[0]], points_y[sel_edge[1]]], color='blue')
            print(f"Coordinates edge:{sel_edge} vertex1: x={points_x[sel_edge[0]]} y={points_y[sel_edge[0]]}, vertex2: x={points_x[sel_edge[1]]} y={points_y[sel_edge[1]]}")

    # Display the plot with the instance and its solution
    if uid_found and not uid_changed:
        # Set titles and labels for the subplots
        axs.set_title("Instance Points and Solution Edges")
        axs.set_xlabel("X Coordinates")
        axs.set_ylabel("Y Coordinates")
        display(fig)
    fig.clear()
    axs.clear()

widgets.interact(interactive_plot_solutions, sel_uid = uid_list, sel_edge = edges_list)

#[print(f"Triangulation:{tr}") for tr in triang] # Print all triangulations


interactive(children=(Dropdown(description='UID', layout=Layout(height='60px%', width='50%'), options=('cgshop…

<function __main__.interactive_plot_solutions(sel_uid, sel_edge)>

# Calculate and plot triangulations

Initialize variables to store the **solution edges (sol_edges)** of the selected instance UID and the calculated **triangulations (triang)**.

In [7]:
# Init variables to be used as global
sol_edges = []
triang = []

User interface initialization

In [ ]:
# Init user interface widgets
uid_list,triang_list = ui.set_ui_elements_triangulations(idb,triang)

Function definition to discard undesired triangulations.

<img src="media/discard_triangs.png" alt="Fig" title="Figure 1. Nested triangulations" width="300"/>

The algorithm to find triangulations is adding undesired solutions. The image above show an example of wrong triangulations in red {ABE},{ABD} and good triangulations in green {ABC}. Then it was necessary to implement the next algorithm to discard outer triangulations. In the example triangulations closed by vertices E and D must be removed. The proposed solution for this problem is execute the convex hull algorithm over nested triangulations points, in this way we can find the undesired vertices. For example the first convex hull will return as solution the vertices A,B and E, as we know that A and B forms the reference edge it can be discarded vertex E, this steps are repeated until just 3 vertices remains A,B and C.

In [9]:
# Algorithm to discard the outer vertex on nested triangulations
def discard_nested_triangs(side_vertices,points_x,points_y,i):
    global triang
    global sol_edges
    points = [] # List all points to apply convex hull algorithm
    points.append([points_x[sol_edges[i][0]],points_y[sol_edges[i][0]]])
    points.append([points_x[sol_edges[i][1]],points_y[sol_edges[i][1]]])
    while len(side_vertices) > 1:
        for vx in side_vertices:
            points.append([points_x[vx],points_y[vx]])
        #print(f"CCW Points: {points}")
        hull = ConvexHull(points)
        breaker = False 
        for hll in hull.vertices:
            for ccw_vx in side_vertices:
                #print(f"Points hull : {points[hll]} ccw_points{[points_x[ccw_vx],points_y[ccw_vx]]}")
                if points[hll] == [points_x[ccw_vx],points_y[ccw_vx]]:
                    side_vertices.remove(ccw_vx)
                    #print(f"Point to discard {ccw_vx} new length: {len(ccw_vertex)}")
                    breaker = True
                    break
            if breaker:
                break
    # Add triangulations 
    if len(side_vertices) == 1:
        e_i = [sol_edges[i][0],side_vertices[0]] # Get edges i,j from point in ccw side
        e_j = [sol_edges[i][1],side_vertices[0]]
        triang.append([frozenset(sol_edges[i]),e_i,e_j])  # Add triangulations found on the list
        #print(f"Triang appended:{[frozenset(sol_edges[i]),e_ccw_i,e_ccw_j]}")
    else:
        if len(side_vertices) != 0:
            print(f"Somethign wrong on vertex {side_vertices}")

Function to calculate determinat.

In [10]:
def det(p1, p2, p3):
    """ 
    > 0: CCW turn
    < 0 CW turn
    = 0: colinear
    """
    return (p2[0] - p1[0]) * (p3[1] - p1[1]) -(p2[1] - p1[1]) * (p3[0] - p1[0])

Function to calculate triangulations.

<img src="media/edges_step_1.png" alt="Fig" title="" width="400"/>

First step to find triangulations is, taking an edge as reference, find all the edges having a vertex in common with one of the vertex of the reference edge, the image above shows all the  edges that is expected to found.

Second step, with the edges found now we need to find the pair of edges closing the triangulations comparing all the edges ***e_i*** against edges ***e_j*** to obtain the ones sharing one vertex.

To find all triangulatios repeat both steps over all edges.


In [11]:
def find_triangulations(points_x,points_y):
    global sol_edges
    global triang
    # Array for triangulations [[e1,e2,e3],[e4,e5,e6], ... [...]]
    triang = []
    
    # Iterate over all solution edges
    for i in range(len(sol_edges)): 
        e_i = set() # Edges matching first vertex 
        e_j = set() # Edges matching second vertex
        #print(f"e_[{i}]: [{sol_edges[i][0]},{sol_edges[i][1]}] ") # Print the soluction edge to be comparted with rest of the edges
        for j in range(i+1,len(sol_edges)):
            if(sol_edges[i][0] == sol_edges[j][0] or sol_edges[i][0] == sol_edges[j][1]): # Add all edges matching with vertex i 
                e_i.add(frozenset(sol_edges[j])) 

            if(sol_edges[i][1] == sol_edges[j][1] or sol_edges[i][1] == sol_edges[j][0]): # Add all edges matching with vertex j
                e_j.add(frozenset(sol_edges[j]))

        # Clear lists
        ccw_vertex = [] 
        cw_vertex = []
        current_triangs = []
        aux = []
        # Compare all vertex in e_i and e_j to find the pair of edges with third vertex coincidence
        for ei in e_i:
            for ej in e_j:
                if ei & ej:
                    aux = ei & ej # Intersection operation for sets. Length of the result will be 1 if the list ei and ej shares the value of one of his elements
                if len(aux) == 1: 
                    current_triangs.append([sol_edges[i],ei,ej])  # Add triangulation found on the list
                    #print(f"Triang:{[sol_edges[i],ei,ej]} 3rd vertex: {list(aux)[0]} ") 
                    ref_vertx_i = [points_x[sol_edges[i][0]],points_y[sol_edges[i][0]]] # Get vertex coordinates from reference edge
                    ref_vertx_j = [points_x[sol_edges[i][1]],points_y[sol_edges[i][1]]] # Get vertex coordinates from reference edge
                    vertx_to_check = [points_x[list(aux)[0]],points_y[list(aux)[0]]] # Get vertex coordinates that close the triangulation
                    side = det(ref_vertx_i,ref_vertx_j,vertx_to_check) # Get side location of 3rd vertex 
                    if side > 0: # CCW 
                        ccw_vertex.append(list(aux)[0]) # Append vertex on CCW side
                    elif side < 0: # CW
                        cw_vertex.append(list(aux)[0]) # Append vertex on CW side
                    else:
                        print(f"No side detected, det = {side} on triangulation {[sol_edges[i],ei,ej]} ") # If this line is called something is wrong
                aux = [] # Clear aux variable
        
        discard_nested_triangs(ccw_vertex,points_x,points_y,i) # Discard nested triangulations on CCW side if there are, if not just add triangulation
        discard_nested_triangs(cw_vertex,points_x,points_y,i) # Discard nested triangulations on CW side if there are, if not just add triangulation

Function to graph solution and triangulations interactively based on selected instance UID and selected triangulation.

In [12]:
# Init variable to be used as global
prev_uid = None

# Interactive function that will be called whith changes on selection widgets(UID or Triangulation)
def interactive_plot_triangulations(sel_uid,sel_triang):
    # Init plot
    plt.ioff()
    fig,(axs1,axs2) = plt.subplots(2,figsize=(10,10))

    # Global variables
    global triang_list
    global prev_uid
    global triang
    global sol_edges

    # UID change detector
    uid_changed = False
    if sel_uid != prev_uid:
        uid_changed = True
        prev_uid = sel_uid

    #  Find instance with uid
    points_x = []
    points_y = []
    uid_found = False
    for instance in idb:
        if(instance.instance_uid == sel_uid):
            uid_found = True
            points_x = instance.points_x
            points_y = instance.points_y
            # Plot instance points
            #visualization.plot_instance(axs,instance) 
            break

    if uid_found == False : 
        print("uid not found")
        #exit()instance_selected = 0
    
    # Find solution with iud
    uid_found = False
    sol_edges = []
    for solution in ZipSolutionIterator("example_solutions.zip"):
        if(solution.instance_uid == sel_uid):
            uid_found = True
            sol_edges = solution.edges
            break
    
    # Logic to find triangulations
    find_triangulations(points_x,points_y)
    
    # Update triangulation list on selection widget
    triang_list,sel_triang = ui.update_triang_list(uid_changed,triang_list,triang,sel_triang)
    
    # Plot all triangulations, to confirm all expected triangulations were obtained
    if uid_found:
        for edge in sol_edges: 
            axs2.plot([points_x[edge[0]], points_x[edge[1]]], [points_y[edge[0]], points_y[edge[1]]], 'r-')
        for tr in list(triang):
            for edge in tr:
                axs2.plot([points_x[list(edge)[0]], points_x[list(edge)[1]]], [points_y[list(edge)[0]], points_y[list(edge)[1]]], color='blue')
    
    # Display plot
    if uid_found and not uid_changed:
        axs2.set_title("Triangulation verification, discarting nested triangs")

    if uid_found:
        for edge in sol_edges:
            axs1.plot([points_x[edge[0]], points_x[edge[1]]], [points_y[edge[0]], points_y[edge[1]]], 'r-')
    if sel_triang != None:
        for edge in list(sel_triang):
            axs1.plot([points_x[list(edge)[0]], points_x[list(edge)[1]]], [points_y[list(edge)[0]], points_y[list(edge)[1]]], color='blue')
            
    # Display the plot with the instance and its solution
    if uid_found and not uid_changed:
        # Set titles and labels for the subplots
        axs1.set_title("Instance Points and Solution Edges")
        axs1.set_xlabel("X Coordinates")
        axs1.set_ylabel("Y Coordinates")
        display(fig)
    fig.clear()
    axs1.clear()
    axs2.clear()


In [ ]:
widgets.interact(interactive_plot_triangulations, sel_uid = uid_list, sel_triang = triang_list)

interactive(children=(Dropdown(description='UID', layout=Layout(height='60px%', width='50%'), options=('cgshop…

<function __main__.interactive_plot_triangulations(sel_uid, sel_triang)>

# Verify Solutions
Verify the solutions using the verify function from cgshop2025_pyutils and print the results.

In [14]:
# Verify Solutions

# Verify the solutions using the verify function from cgshop2025_pyutils and print the results
for solution in ZipSolutionIterator("example_solutions.zip"):
   instance = idb[solution.instance_uid]  # Retrieve the instance corresponding to the solution
   result = verify(instance, solution)  # Verify the solution against the instance
   print(f"{solution.instance_uid}: {result}")  # Print the verification result
   assert not result.errors, "Expect no errors."  # Ensure there are no errors in the verification result

cgshop2025_examples_simple-polygon-exterior-20_60_5da99fdb: num_obtuse_triangles=53 num_steiner_points=0 errors=[]
cgshop2025_examples_ortho_20_b099d1fe: num_obtuse_triangles=11 num_steiner_points=0 errors=[]
cgshop2025_examples_simple-polygon-exterior-20_40_97467655: num_obtuse_triangles=48 num_steiner_points=0 errors=[]
cgshop2025_examples_point-set_40_16c501a5: num_obtuse_triangles=45 num_steiner_points=0 errors=[]
cgshop2025_examples_simple-polygon-exterior-20_250_329290ff: num_obtuse_triangles=244 num_steiner_points=0 errors=[]
cgshop2025_examples_simple-polygon-exterior_40_3df495b2: num_obtuse_triangles=41 num_steiner_points=0 errors=[]
cgshop2025_examples_simple-polygon-exterior-20_80_23272e96: num_obtuse_triangles=89 num_steiner_points=0 errors=[]
cgshop2025_examples_simple-polygon-exterior_60_82e60438: num_obtuse_triangles=60 num_steiner_points=0 errors=[]
cgshop2025_examples_simple-polygon-exterior-20_150_b988b15c: num_obtuse_triangles=169 num_steiner_points=0 errors=[]
cgsho

Esto es una nueva atualización 